# Secure Hash Standard: Binary Word Operations — Problem 1

## Introduction

This notebook is for the comp-theory module problems where I will document my progress .  
Problem 1 focuses on implementing a set of 32-bit word operations used in the **Secure Hash Standard (SHA-256)**.

These operations include logical functions such as `Ch`, `Maj`, and `Parity`, along with the rotation/shift functions  
`Σ0`, `Σ1`, `σ0`, and `σ1`.

All functions will be implemented in Python using **NumPy**, ensuring that all values behave as unsigned 32-bit integers  
(`np.uint32`). Each function will include:

- a clear docstring  
- a Markdown explanation  
- correct 32-bit behaviour  
- test examples verifying correctness  




## Background

The Secure Hash Standard (NIST FIPS 180-4) defines a series of operations on 32-bit words used by the SHA-256  
compression function. These operations must behave like 32-bit unsigned integers, including wraparound  
behaviour.

### Definitions (FIPS 180-4, Section 4.1.2)

Logical functions:
- **Parity(x, y, z)** = x XOR y XOR z  
- **Ch(x, y, z)** = (x AND y) XOR ((NOT x) AND z)  
- **Maj(x, y, z)** = (x AND y) XOR (x AND z) XOR (y AND z)

Big sigma functions:
- **Σ0(x)** = ROTR²(x) XOR ROTR¹³(x) XOR ROTR²²(x)  
- **Σ1(x)** = ROTR⁶(x) XOR ROTR¹¹(x) XOR ROTR²⁵(x)

Small sigma functions:
- **σ0(x)** = ROTR⁷(x) XOR ROTR¹⁸(x) XOR SHR³(x)  
- **σ1(x)** = ROTR¹⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

Before implementing these, we create helper functions for rotation and logical right shift.


In [8]:
import numpy as np


### Helper Functions: ROTR and SHR

SHA-256 relies on two low-level bit operations:

- **Rotate Right (ROTR)** — circular rotation of x by n bits  
- **Logical Right Shift (SHR)** — shift right, filling with zeroes  

These helper functions ensure correct 32-bit wraparound behaviour.


In [9]:
def rotr(x: np.uint32, n: int) -> np.uint32:
    """
    Rotate Right (ROTR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to rotate.

    Returns
    -------
    np.uint32
        The rotated 32-bit result.
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x: np.uint32, n: int) -> np.uint32:
    """
    Logical Right Shift (SHR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to shift.

    Returns
    -------
    np.uint32
        The shifted value (zero-filled).
    """
    return np.uint32(x >> n)


### Logical Functions: Parity, Ch, Maj

These functions operate on three 32-bit words.  
They implement core decision and mixing behaviours used by the SHA-256 compression loop.


In [10]:
def Parity(x, y, z):
    """
    Parity function for SHA-256.
    Computes x XOR y XOR z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32(x ^ y ^ z)


def Ch(x, y, z):
    """
    Choice function.
    For each bit of x: if the bit is 1, choose y; otherwise choose z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (~x & z))


def Maj(x, y, z):
    """
    Majority function.
    For each bit, returns the majority value among x, y, z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


### SHA-256 Sigma Functions

These operations mix input bits using rotations and shifts.  
They form the core of the SHA-256 message schedule and compression function.


In [11]:
def Sigma0(x):
    """Big Sigma 0: ROTR2 ^ ROTR13 ^ ROTR22"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    """Big Sigma 1: ROTR6 ^ ROTR11 ^ ROTR25"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


def sigma0(x):
    """Small sigma 0: ROTR7 ^ ROTR18 ^ SHR3"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    """Small sigma 1: ROTR17 ^ ROTR19 ^ SHR10"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


### Testing the Functions

We now apply simple tests to verify correct behaviour of all functions.


In [12]:
x = np.uint32(0x12345678)

print("Sigma0 =", hex(Sigma0(x)))
print("Sigma1 =", hex(Sigma1(x)))
print("sigma0 =", hex(sigma0(x)))
print("sigma1 =", hex(sigma1(x)))

a = np.uint32(0xFFFFFFFF)
b = np.uint32(0x00000000)
c = np.uint32(0xAAAAAAAA)

print("Ch(a,b,c) =", hex(Ch(a,b,c)))
print("Maj(a,a,a) =", hex(Maj(a,a,a)))
print("Parity(x,x,x) =", hex(Parity(x,x,x)))


Sigma0 = 0x66146474
Sigma1 = 0x3561abda
sigma0 = 0xe7fce6ee
sigma1 = 0xa1f78649
Ch(a,b,c) = 0x0
Maj(a,a,a) = 0xffffffff
Parity(x,x,x) = 0x12345678
